In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def getAcc( pos, mass, G, softening ):
	"""
    Calcola l'accelerazione di ogni particella dovuta alla forza di Newton
	pos  è una matrice del tipo N x 3 di posizioni
	mass è un vettore di tipo N x 1 di masse
	G è la costante di gravitazione di Newton
	softening è la softening length
	a è una matrice del tipo N x 3 di accelerazioni
	"""
	# posizioni r = [x,y,z] per tutte le particelle:
	x = pos[:,0:1]
	y = pos[:,1:2]
	z = pos[:,2:3]

	# matrice che contiene tutte le separazioni di particelle a coppie: r_j - r_i
	dx = x.T - x
	dy = y.T - y
	dz = z.T - z

	# matriche che contiene 1/r^3 per tutte le separazioni di particelle a coppie:
	inv_r3 = (dx**2 + dy**2 + dz**2 + softening**2)
	inv_r3[inv_r3>0] = inv_r3[inv_r3>0]**(-1.5)

	ax = G * (dx * inv_r3) @ mass
	ay = G * (dy * inv_r3) @ mass
	az = G * (dz * inv_r3) @ mass
	
	# riunisce le componenti della velocità:
	a = np.hstack((ax,ay,az))

	return a
	
def getEnergy( pos, vel, mass, G ):
	"""
	Calcola l’energia cinetica (KE) e potenziale (PE) del sistema
	"""
	# Energia cinetica:
	KE = 0.5 * np.sum(np.sum( mass * vel**2 ))


	# Energia potenziale:
	x = pos[:,0:1]
	y = pos[:,1:2]
	z = pos[:,2:3]

	dx = x.T - x
	dy = y.T - y
	dz = z.T - z

	# matrice che contine 1/r per tutte le separazioni delle particelle a coppie:
	inv_r = np.sqrt(dx**2 + dy**2 + dz**2)
	inv_r[inv_r>0] = 1.0/inv_r[inv_r>0]

	# la somma avviene in modo che ogni interazione venga contata una sola volta
	PE = G * np.sum(np.sum(np.triu(-(mass*mass.T)*inv_r,1)))
	
	return KE, PE;


def main():
	""" simulazione effettiva """
	
	# Variabili della simulazione
	N         = 50     # numero di particelle
	t         = 0      # istante di inizio
	tEnd      = 10.0   # istante di fine
	dt        = 0.01   # minor dt possibile
	softening = 0.1    # softening length
	G         = 1      # costante di gravitazione di Newton, da cambiare a "6.67408 × 1e-11" se si vogliono le misure reali
	plotRealTime = True # fornisce i grafici in tempo reale
	
	# Genera le restanti condizioni iniziali per le particelle
	np.random.seed(17)            # imposta il seme per il generatore casuale
	
	mass = 20.0*np.ones((N,1))/N  # la massa totale nel sistema è 20
	pos  = np.random.randn(N,3)   # seleziona casualmente posizioni e velocità
	vel  = np.random.randn(N,3)
	
	# calcola uno schema a centro di massa
	vel -= np.mean(mass * vel,0) / np.mean(mass)
	
	# calcola le accelerazioni gravitazionali iniziali
	acc = getAcc( pos, mass, G, softening )
	
	# calcola l’energia del sistema
	KE, PE  = getEnergy( pos, vel, mass, G )
	
	# numero di dt
	Nt = int(np.ceil(tEnd/dt))
	
	# salva posizioni ed energie per realizzare le scie nel grafico
	pos_save = np.zeros((N,3,Nt+1))
	pos_save[:,:,0] = pos
	KE_save = np.zeros(Nt+1)
	KE_save[0] = KE
	PE_save = np.zeros(Nt+1)
	PE_save[0] = PE
	t_all = np.arange(Nt+1)*dt
	
	# preparare il grafico impostando le assi e il resto
	fig = plt.figure(figsize=(4,5), dpi=80)
	grid = plt.GridSpec(3, 1, wspace=0.0, hspace=0.3)
	ax1 = plt.subplot(grid[0:2,0])
	ax2 = plt.subplot(grid[2,0])
	
	# loop principale in cui si utilizza il metodo kick-drift-kick
	for i in range(Nt):
		# primo mezzo kick
		vel += acc * dt/2.0
		
		# drift
		pos += vel * dt
		
		# aggiorna le accelerazioni
		acc = getAcc( pos, mass, G, softening )
		
		# secondo mezzo kick
		vel += acc * dt/2.0
		
		# aggiorna il tempo
		t += dt
		
		# ottieni l’energia del sistema
		KE, PE  = getEnergy( pos, vel, mass, G )
		
		# salva le energie e le altre variabili per creare le scie sul grafico
		pos_save[:,:,i+1] = pos
		KE_save[i+1] = KE
		PE_save[i+1] = PE
		
		# realizza i grafici in tempo reale
		if plotRealTime or (i == Nt-1):
			plt.sca(ax1)
			plt.cla()
			xx = pos_save[:,0,max(i-50,0):i+1]
			yy = pos_save[:,1,max(i-50,0):i+1]
			plt.scatter(xx,yy,s=1,color=[.7,.7,1])
			plt.scatter(pos[:,0],pos[:,1],s=10,color='blue')
			ax1.set(xlim=(-2, 2), ylim=(-2, 2))
			ax1.set_aspect('equal', 'box')
			ax1.set_xticks([-2,-1,0,1,2])
			ax1.set_yticks([-2,-1,0,1,2])
			
			plt.sca(ax2)
			plt.cla()
			plt.scatter(t_all,KE_save,color='red',s=1,label='KE' if i == Nt-1 else "")
			plt.scatter(t_all,PE_save,color='blue',s=1,label='PE' if i == Nt-1 else "")
			plt.scatter(t_all,KE_save+PE_save,color='black',s=1,label='Etot' if i == Nt-1 else "")
			ax2.set(xlim=(0, tEnd), ylim=(-300, 300))
			ax2.set_aspect(0.007)
			
			plt.pause(0.001)
	    
	
	
	# aggiunge la legenda
	plt.sca(ax2)
	plt.xlabel('time')
	plt.ylabel('energy')
	ax2.legend(loc='upper right')
	
	# salva l’immagine
	plt.savefig('nbody.png',dpi=240)
	
	
	    
	return
	


  
if __name__== "__main__":
  main()